# Running the CaImAn pipeline with DataJoint

This notebook provides the necessary steps to run the CaImAn segmentation pipeline on the DataJoint database. 

<b>IMPORTANT: </b> This notebook assumes that all user-specific data (for `common_exp.Session()`, `common_img.Scan()` and `common_img.RawImagingFile()`) have been entered elsewhere! Once this is done, the pipeline is common for all useres and can be executed here.

Because this notebook is using CaImAn's functions, it needs to be run inside the Caiman environment. If you are in the Datajoint environment, the tables will still load and you can view the data, but trying to call Caiman-specific functions will result in `ModuleNotFoundErrors` or `NameErrors`.

In [1]:
# Importing datajoint tables
import sys
sys.path.append("..\\")

import login
login.connect()
from schema import common_img, common_exp
import datajoint as dj

Connecting hheise@130.60.53.48:3306


## Step 0: Check that the manual data is there

Before running the actual Caiman pipeline, the manual data has to be entered into the database so that Datajoint knows where to start looking for the data. You can do this by checking the `Session()` and `Scan()` tables, with restrictions to only get the sessions you are about to process.

In [2]:
# Restrict the query with some parameters
restrictions = dict(username='hheise', mouse_id=69)

# Show the entries of the tables that fit these restrictions
(common_exp.Session() & restrictions & "day = '2021-08-11'")

"username Unique username, should be the UZH shortname",mouse_id ID of mouse (unique per investigator),day Date of the experimental session (YYYY-MM-DD),session_num Counter of experimental sessions on the same day (base 1),session_id Unique identifier,session_path Relative path of this session on the Neurophysiology-Storage1 server,session_counter Overall counter of all sessions across mice (base 0),"experimenter Who actually performed the experiment, must be a username from Investigator()",anesthesia Anesthesia short name,setup Unique name of the setup,task Unique name of the task,session_notes description of important things that happened


In [ ]:
common_img.Scan() & restrictions & "day = '2021-07-16'"

## Run the pipeline: Populate Computed Tables

If the above queries returned the right data, we are good to go for our pipeline!

Datajoint's pipeline consists mainly of consecutive `Computed` tables, that can be filled in automatically with the processed data. This "filling in" is done by calling each table's `populate()` function. A detailed explanation can be found [here](https://docs.datajoint.org/python/computation/01-autopopulate.html#populate), but in short, Datajoint looks up all possible combinations of entries in upstream tables and runs it's `make()` function (where the actual computation for the table is located) for each of these combinations. This makes it possible for example to run and store the Caiman Segmentation for one session and several sets of parameters.

Datajoint offers us a basic way to visualize the tables and its dependencies. The color of each table name shows its type (green = `Manual`, grey = `Lookup`, blue = `Imported`, red = `Computed`). The pipeline is visible as the "string" of red `Computed` tables going from `ScanInfo()` to `Deconvolution()`.

In [ ]:
# Draw the Entity Relationship Diagram (ERD) of a schema.
### If the error '[WinError 2] "dot" not found in path.' occurs, you have to install graphviz by running 
### "conda install -c conda-forge python-graphviz" in an anaconda prompt with your environment active.
%matplotlib qt
dj.ERD(common_img).draw()

## Step 0.5: Populate RawImagingFile()

Before running the actual pipeline, we have to find the raw imaging files from the current scan session. Datajoint does this automatically, but makes two assumptions that each user has to ensure is true:

1. All imaging files for each session have to be located in the directory specified in `session_path` on the user's data directory on the Wahl server
2. All imaging files have to follow the naming convention(s) specified in the user's `gui_params.yaml` file under the `scientifica_file` entry

Datajoint then goes through the `session_path` directory and all its subfolders and finds all files that fit the pattern. The pipeline will then be performed on these files.

**Important:** The database only stores the **file paths** of raw files, not the files themselves, due to memory constraints. It cannot keep track of changes to this file path after you entered the data into `RawImagingFile()`. This means that if you move the files to a different location, or rename one of the upstream folders, you will break the file path and Datajoint will not find the files anymore. In this case, if you want to perform processing steps where Datajoint needs access to the raw files (like motion correction or segmentation), you will have to update `session_path` of these files before.

In [ ]:
# Populate the table
common_img.RawImagingFile().populate(display_progress=True)

# View the resulting entry
common_img.RawImagingFile()

## Step 1: Populate ScanInfo()

The first `Computed` table we have to populate is `ScanInfo()`. This table stores hardware and software settings of the scan that can be automatically read from the raw TIFF files coming from ScanImage, like frame rate, Pockels cell power, or XYZ stage position.

In [ ]:
# Populate the table
common_img.ScanInfo().populate({'username': 'hheise'}, reserve_jobs=False, display_progress=True)

# View the resulting entry
common_img.ScanInfo() & restrictions

## Step 2: Motion Correction

Next up is the motion correction. Here we need a parameter set for the first time. The parameters for the motion correction is stored in the `Manual` table `MotionParameter()`. Because the motion correction parameters are generally transferable across brain regions and mice, we have a single table storing all sets. Each has a short description about the condition for which this set is appropriate (e.g. which Zoom setting, low or high movement artefacts, etc).

Let's have a look:

In [ ]:
common_img.MotionParameter()

If we want to add a parameter set, we can do this manually by entering a new dict into the table. This function also automatically saves manual each entry in a backup YAML file, in case the database gets corrupted and has to be restored.

If you do not want to change each parameter, you can comment them out. In this case, the default value from `motion_id = 0` will be used.

In [ ]:
new_entry = dict(
                 motion_id=0,
                 motion_shortname='default',
                 motion_description='Default values that work well from Hendriks previous experience. Uses piecewise-rigid correction.',
                 crop_left=12,
                 crop_right=12,
                 offset=220,
                 max_shift=50,
                 stride_mc=150,
                 overlap_mc=30,
                 pw_rigid=1,
                 max_dev_rigid=3,
                 border_nan=0,
                 n_iter_rig=2,
                 nonneg_movie=1,
                )

common_img.MotionParameter().helper_insert1(new_entry)

Once we have decided which parameter set to use, we can call the `populate()` function of the `MotionCorrection()` table with the respective restriction.

`MotionCorrection()` does several things for each scan:
1. Download the raw TIFF files for this scan into a temporary cache directory on the local machine.
2. Do some preprocessing necessary for files from the H37R Scientifica: Align odd and even scan lines, crop the left and right borders to avoid edge artifacts, and apply a fixed offset (see `MotionParameter()` entry) to avoid mean negative pixel values that can mess with the CNMF algorithm.
3. Perform the actual motion correction.
4. Extract the calculated pixel shifts from the motion correction and store it in the database.
5. Calculate and save some quality control metrics (e.g. correlation with template, means and standard deviations)
6. Delete the memory-mapped files from the disk.

As you can see, we delete the memory-mapped file directly after the motion correction. This is because we only store the pixel shifts in the database, and not the entire file. When we need the motion-corrected memory-mapped movie, we can recreate it easily by applying the pixel shifts to the corrected raw TIFF files. This is much faster than running the entire motion correction algorithm again, and it saves tons of disk space on the server.

In [ ]:
# Call the populate() function, restricting it if necessary for only one set of parameters
common_img.MotionCorrection().populate({'username': 'hheise', 'motion_id': 0, 'mouse_id': 69}, display_progress=True)

# View the resulting entry
common_img.MotionCorrection() & restrictions

## Step 2.5: Quality control of MotionCorrection()

To be able to quickly judge whether the motion correction was sufficiently effective without having to export the entire movie, we are computing some metrics that can help evaluate the motion correction. This table is not necessary for the next step of the pipeline, but is nice to have.

These are Z-projections where each pixel has a certain value dependent on the time course of this pixel. The Z-projections we calculate are:
- Average intensity
- Local correlation (intensity correlation between each pixel and its 8 neighbors)
- Standard deviation, minimum and maximum
- 99.9th percentile
- average intensity of all pixels over time (makes slow intensity changes visible)

In [ ]:
common_img.QualityControl().populate({'username': 'hheise'}, reserve_jobs=False, display_progress=True)

common_img.QualityControl() & restrictions

## Step 3: Segmentation

This is the actual CaImAn segmentation algorithm. It performs component detection, evaluation and dF/F computation steps. For this we also need a parameter set, which are stored in `CaimanParameter()`. Because these parameters are much more dependent on brain region and the specific FOV of the mouse (cell density, fluorescence intensity, etc.), each mouse gets its own list of parameter sets. If you want to use the same set of parameters for several mice, the set has to be entered into the database for each mouse. 

This means that instead of only one ID (like `motion_id`), `CaimanParameter()` needs two, `caiman_id` and `mouse_id`. This also means that several mice can have a set with `caiman_id=0`, but that contains slightly different parameters. This can be useful to store sets with different criteria. `caiman_id=0` can be a strict parameter set that only finds 100% certain neurons, and `caiman_id=1` could be more lenient, also accepting components that are not 100% neurons. The exact values in each set will vary between mice, but the "meaning" of each `caiman_id` is conserved across mice.

In [ ]:
common_img.CaimanParameter()

In [ ]:
new_entry = dict(
            # Identifiers
             username = 'hheise',
             mouse_id = 93,
             caiman_id = 0,
            # Component detection parameters
             p = 1,
             nb = 2,
             merge_thr = 0.75,
             rf = 25,
             stride_cnmf = 6,
             k = 18,
             g_sig = 4,
             method_init = 'greedy_roi',
             ssub = 2,
             tsub = 2,
            # Evaluation parameters
             snr_lowest = 5.0,
             snr_thr = 9.0,
             rval_lowest = -1.0,
             rval_thr = 0.85,
             cnn_lowest = 0.1,
             cnn_thr = 0.9,
            # dF/F parameters
             flag_auto = 1,
             quantile_min = 8,
             frame_window = 2000,
            )

common_img.CaimanParameter().helper_insert1(new_entry)

The `Segmentation()` table performs the complete CaImAn pipeline. Results are not stored in an HDF5 file (although you can save them as such), but stored directly in the database. Each accepted component and their attributes (spatial and temporal components, dF/F, evaluation scores) are stored in the `Segmentation.ROI()` Part-table.

`Segmentation().populate()` has two additional arguments, which control whether a FOV overview with outlined components should be saved after source detection and after evaluation (`save_overviews`) and whether the results should be stored in a traditional `cnm_results.hdf5` file in addition to the database entries (`save_results`). If `True`, these results are saved in the session folder on the Wahl server.

In [4]:
common_img.Segmentation().populate({'username': 'hheise', 'caiman_id': 0}, make_kwargs=dict(save_overviews=True, save_results=False), display_progress=True)

common_img.Segmentation() & restrictions

Segmentation:   0%|                                                                                                                                                                        | 0/31 [00:00<?, ?it/s]

Populating Segmentation for {'username': 'hheise', 'mouse_id': 69, 'day': datetime.date(2021, 3, 10), 'session_num': 1, 'motion_id': 0, 'caiman_id': 0}.
Creating memory mapped file...


Segmentation:   6%|██████████▏                                                                                                                                                  | 2/31 [15:16<3:41:36, 458.49s/it]


DataJointError: fetch1 should only return one tuple. 0 tuples found

In [ ]:
common_img.Segmentation.ROI()

## Step 4: Deconvolution

The last step of our pipeline is Peter's CASCADE deconvolution. For this we need to decide which pretrained model to use. The best four are included in `DeconvolutionModel()`, with different parameters that influence the deconvolution [described here](https://github.com/HelmchenLabSoftware/Cascade#what-does-the-smoothing-parameter-for-the-models-mean):

- Smoothing window: standard deviation of smoothing Gaussian kernel around ground truth in milliseconds. Depends on the frame rate of the recording. At 30 Hz, a smoothing of 50 ms is recommended. If imaging quality is bad, increase smoothing window.
- Causal kernel: By default, the ground truth is smoothed symmetrically in time with a Gaussian kernel. This might predict some activity before the actual calcium transient. If time resolution is very important, e.g. with stimulus-triggered activity, a causal kernel almost exclusively predicts activity after the calcium event. However, a causal kernel is more error-prone and might predict events on noise, so should only be chosen if really necessary.

Currently, four models are included in `DeconvolutionModel()`: 50ms, 100ms, causal kernel, Gaussian kernel. Additional models can be downloaded [here](https://github.com/HelmchenLabSoftware/Cascade/blob/master/Pretrained_models/available_models.yaml) and added to the table. In general, the ensemble models trained on many different datasets of excitatory activity (labeled `Global_EXC_`) perform better than single models and should be preferred.

When populating `Deconvolution()`, restrict the algorithm to only use the most suitable model.

The results of of the deconvolution are stored in the `Deconvolution.ROI()` part table again, with the same ID as in `Segmentation.ROI()`.

In [ ]:
common_img.DeconvolutionModel()

In [ ]:
common_img.Deconvolution().populate({'username': 'hheise', 'decon_id': 1}, display_progress=True, reserve_jobs=True)

## Step 4.5: Activity Statistics

Lastly, `ActivityStatistics()` computes and stores some analysis of the deconvolution traces like number of spikes, average spike rate, and number of events (threshold crossings, one event can be more than one spike).

In [ ]:
common_img.ActivityStatistics().populate({'username': 'hheise'}, reserve_jobs=True, display_progress=True)

In [ ]:
from matplotlib import pyplot as plt

spikerate = (common_img.ActivityStatistics.ROI() & restrictions).fetch('rate_spikes')

plt.hist(spikerate)